<img src="readme.png" alt="Schematic diagram of DigNet generation network" />

## 1.输入数据
### 1.1 
基因表达谱和GRN(数据格式见`Supplement S1`)

- **'net' variable**: Contains the adjacency matrix of network data, a 0-1 weight matrix (Numpy ndarray). Non-0-1 values can also be loaded, sized cell * cell.
- **'exp' variable**: Contains experimental data in DataFrame format, preprocessed scRNA-seq results (CSV format), sized gene * cell.


### 1.2
只输入基因表达谱, 有方法构建一个参考网络(见manuscript, 即readme文档使用的输入格式), 接下来都是使用该格式输入

预处理:参考`CancerDatasets/Create_BRCA_data.py`,基因表达谱是一个csv或xlsx文件,第一行是cell numbers, 第一列是gene symbol IDs



## 2.文件和文件夹

- `pre_train/`: The pre-trained DigNet model provided in this article
- `Cancer_datasets/`: Contains preprocessed results of the E-MTAB-8107 data used in the manuscript
>预处理后的数据,但是在pathway/中使用的是预处理前的数据.预处理前应该是csv格式,预处理后应该是mat,data格式.根据解压出的是csv,判断该文件夹下其实是预处理前的数据

- `pathway/`: sub-function file containing data preprocessing

> 预处理
> MaxMinNormalization(x, Min=0, Max=1): 归一化到(0,1)区间

> cal_percent(new_bit_crop,corr_TF_Gene, MI_TF_Gene,net_bit_orig):<br>
1.首先new_bit_origC赋值为net_bit_orig和的new_bit_crop交集,然后计算在new_bit_crop中的比例,记为NUM_ORIG<br>
2.corr_TF_GeneC赋值为corr_TF_GeneC和new_bit_crop的交集,然后计算与net_bit_origC不相交的个数在new_bit_crop中的比例,记为NUM_PCC<br>
3.对MI_TF_Gene的操作与corr_TF_GeneC相同,得到NUM_MI<br>
4.如果NUM_ORIG,NUM_PCC,NUM_MI相加不是100,则缩放使得和为100<br>
5.NUM_PCC+NUM_MIover>50 -> overflow = True<br>
else: overflow = False<br>
return NUM_ORIG, NUM_PCC, NUM_MI, overflow

>load_sergio_count(filename='pathway/simulation/SERGIO_data_node_2000.data',  num=None, logp=True):<br>
用pickle加载filename,如果有num,则选择索引为num的数据,如果logp == True, 则进行log1p变换<br>
return batch

>plot_GRN_percent(network_percent):<br>
画一个"Train_GRN_pecent_bar.pdf"柱形图,<br>
以pathway为横坐标,<br>
NUM_ORIG,NUM_PCC,NUM_MI为纵坐标

>compute_mutual_information(df):<br>
df每行是一个特征<br>
计算df特征间的的mi(互信息)<br>
return mi_matrix

>compare_char(charlist, setlist):<br>
index为setlist中charlist的索引<br>
一般来说index方法只能在list中找一个值的索引,所以charlist可能不是list类型,只是一个值<br>
return index

>calRegnetwork(human_network, GRN_GENE_symbol):<br>
human_network_TF_symbol = human_network第0列<br>
human_network_Gene_symbol= human_network第2列<br>
GRN_GENE_symbol中找调控基因(TF)和靶基因(Gene)<br>
这一段循环方式低效,应该可以改进<br>
return pd.DataFrame(network, columns=['TF', 'Gene'])

>load_KEGG(kegg_file='pathway/kegg/KEGG_all_pathway.pkl'):<br>
如果 pkl 文件存在，则加载它,如果 pkl 文件不存在，则运行 KEGG.py 文件

>high_MI(exp_pca_discretized, exp_pca, net_bit, parm):<br>
计算exp_pca_discretized的互信息,保存在row_MI中,将row_MI的对角线设置为0.然后筛选大于阈值的调控基因和靶基因,把调控基因和靶基因保存到MI_TF_Gene中,并且与net_bit拼接去重.<br>
return net_bit, MI_TF_Gene

>high_pearson(exp_pca, net_bit, parm):<br>
与high_MI中的操作类似,不过未经过离散化.<br>
值得注意的是,mi和pearson相关系数都是对称的,也就是说调控基因和靶基因(有向边)并没有明确的区分?

>from_cancer_create(BRCA_exp_filter_saver, KEGG, parm, lim=200, test_pathway=None, Other_Pathway=None, human_network=None):<br>
1.获得exp表达谱:在KEGG的Other_Pathway中但不在KEGG的test_pathway中<br>
2.对exp进行标准化<br>
3.net_bit,用calRegnetwork函数在human_network的先验信息中得到,由两列组成:调控基因TF和靶基因Gene<br>
4.net_bit_orig:net_bit的拷贝<br>
5.exp_pca:通过pca降维,列数比exp少一<br>
6.将exp_pca分成256个离散化的区间(0-255),保存为exp_pca_discretized<br>
7.用上面两个函数计算高pearson和MI的调控-靶基因,添加到net_bit<br>
8.创建邻接矩阵adj_matrix<br>
8.1 矩阵形状:基因个数(exp.index) * 基因个数<br>
8.2 有调控关系的(TF-Gene)在矩阵中置1<br>
8.3 经过cal_del_TF_edge(exp.index)函数,将exp分成TF,GENE两个不相交的部分<br>
然后在邻接矩阵adj_matrix中将下标为GENE,TF的点置0(这样就从无向图变成了有向图)<br>
9.predicted_adj_matrix, new_graph = pca_cmi(exp_pca_discretized, net_bit, parm['pmi_percent'], 1):<br>
参考**其他文件**,通过删除cmi较小的边,获得邻接矩阵和图.此处有重要的问题,即离散化的数据用多元正态分布的方法计算cmi不合理.<br>
new_bit_crop是更新后的图的所有边<br>
10.以上有两个邻接矩阵adj_matrix,predicted_adj_matrix.前者是基于先验知识(查表)删除回路边,后者是用cmi删除cmi小的边,以下要对两者进行比较<br>
11.分为三种情况,选择边最少的邻接矩阵<br>
11.1 predicted_adj_matrix所有值为0<br>
11.2 (np.sum(adj_matrix) / np.sum(predicted_adj_matrix)) < 0.5:<br>
return exp, adj_matrix, new_row<br>
11.3 else:<br>
return exp, predicted_adj_matrix, new_row<br>
评价:这里用0.5比较,可能是因为adj_matrix是有向图<br>
如果predicted_adj_matrix还是一个对称的无向图,与有向的adj_matrix的边数相同,predicted_adj_matrix会多一倍的边.<br>
但是如果选了predicted_adj_matrix,为什么不把他从对称矩阵(无向)变成有向图?<br>
而且经过复现并不是对称图,这个判断几乎只会选到predicted_adj_matrix,后续可以尝试改成与1比较.

>matrix2Data(adj_matrix, node_feature, num=0, adj2data=True, log_trans=True, metacell=False, Cnum=100, k=20):<br>
1.1如果node_feature是list<br>
node_feature中选择第num个,转换为tensor,得到x<br>
1.2如果node_feature不是list<br>
直接将node_feature转换为tensor,得到x<br>
log_trans默认True,进行对数变换x = log(1+x)<br>
2.归一化:使用MinMaxScalar,得到x_normalized<br>
3.1如果adj2data是True(默认为True),此时邻接矩阵是0-1编码的,即表示无权图<br>
adj_matrix转换为tensor<br>
indices_tensor是所有的边,每一列代表一条边的头节点和尾节点<br>
num_edges是边的数量<br>
values_tensor是一个全为1的张量,长度是边的数量<br>
3.2如果adj2data是False,此时邻接矩阵是一个稠密的,图中每个点(i,j)都有权重的数组<br>
indeces = 邻接矩阵的所有索引,每一列代表一条边的头节点和尾节点<br>
values = 邻接矩阵的所有值<br>
然后转换成张量,indeces_tensor,values_tensor<br>
4.创建data,一个图,节点特征为x_normalized(形状为(节点数,特征数)),边的索引和权重分别是indeces_tensor,values_tensor,索引为0-(节点数-1)<br>
return data

>load_pathway_mat(file_path, num=0):<br>
从mat类型数据中读取data<br>
node_feature是8个节点的特征,这个8是怎么确定的?<br>
adj_matrix是data中的邻接矩阵<br>
Data1 = matrix2Data(adj_matrix, node_feature, num=num)创建图<br>
return Data1

>create_batch_dataset_from_mat(matnum=100, test=False):<br>
1.如果test = True:测试集<br>
直接加载第matnum个mat文件到batch<br>
2.否则:训练集<br>
加载序号为1-matnum的所有mat文件到batch<br>
return batch

>create_batch_dataset_simu(filename='./pathway/simulation/SERGIO_data_node_2000.data', num=None, device=None,test=False,adddata=None, metacell=True, Cnum=100, k=20):<br>
1.如果test == True 测试集<br>
直接加载文件,文件中有邻接矩阵net,基因谱exp,保存到batch<br>
return batch<br>
2.如果test == False 训练集<br>
2.1 文件中有多个net,exp,遍历地取出保存到data_list(相当于一个batch), edge_percent(计算边在图中的比例)<br>
2.2 edge_percent = 所有edge_percent的平均值<br>
2.3 如果有adddata(默认没有), 则在data_list,edge_percent中加入新的数据<br>
2.4 创建batch<br>
2.5 return batch,edge_persent

>cal_metacell(BRCA_exp_filter_saver, Cnum=100, k=20):<br>
BRCA_exp_filter_savert,形状(样本个数, 特征数),具体来说是(细胞个数,基因个数)<br>
K_list,k近邻得到每个细胞的k个邻居,形状(细胞个数,k)<br>
ALL_C_list,长度为细胞个数的数组,元素是基因序号(0-基因个数)<br>
S,元素为None,长度为Cnum的list,用于存储细胞索引<br>
old_S,S的拷贝<br>
Nc_max_list,(1,Cnum)的0数组,用于存储S中对应细胞的最大k近邻<br>
1.while循环:<br>
1.1 ALL_C_list_current,不在S中的基因索引,即一开始是所有的细胞<br>
1.2 对ALL_C_list_current中的c循环,c不在S中<br>
1.2.1 Nc_max,对一个细胞c,所有S中的细胞与他k近邻并集最多的个数<br>
1.2.2 a.如果Nc_max > Nc_max_list中的最小值(初始是0),则对于最小值的位置,S置c,Nc_max_list置Nc_max.其实这个所谓最小值的位置,就是从0递增.<br>
b.elif Nc_max是k*2,即S中存在一个与c的k近邻不交的细胞<br>
对于S中最大值的位置,S置c,Nc_max_list置Nc_max<br>
1.2.3 如果一段时间S没有更新,则结束while<br>
1.3 进行Cnum(即metacell的数量)次循环,每次序号为cn<br>
1.3.1 对于S在cn位置的c细胞,Nc_max是所有S中的细胞与他k近邻并集最多的个数<br>
1.3.2 如果Nc_max>原来位置的值则更新S和Nc_max_list<br>
评价:1.3的循环意义不明,S和Nc_max_list已经在1.2更新过了,这一段似乎完全没用<br>
2.对S进行排序,确认不含None,不含重复值<br>
3.定义BRCA_exp_filter_saver为dataframe<br>
4.对于S中的每个细胞si, 在K_list中取这个细胞的前x个相邻细胞,x为一个metacell中细胞个数,这些细胞每个基因的平均值作为metacell中的基因的值,然后保存到BRCA_exp_filter_saver中,列名为'c' + str(si)<br>
评价:x的计算方式:所有细胞数除metacell个数, 即一个metacell中的细胞数<br>
既然要取最近的x个点,为什么k近邻中近邻数量是20,而不是直接用x(细胞总数/100)<br>
这是为了利用切片的性质,即使x大于20也可以取20个细胞作为一个metacell,不会报错<br>
5.最终得到的BRCA_exp_filter_saver形状为(基因个数,Cnum)<br>
6.return BRCA_exp_filter_saver

>create_batch_dataset_from_cancer(filepath='CancerDatasets/DCA/BRCA_output.csv',test_pathway='hsa05224', test=False, device=None, metacell=True, Cnum=100, k=20, lim=200, return_list=False):<br>
1.1 metacell == True 需要metacell处理<br>
filepath中读取BRCA_exp_filter_saver,数据处理有误,多删了第一列<br>
用函数cal_metacell重新获得BRCA_exp_filter_saver,(基因个数,Cnum= metacell个数)<br>
1.2 metacell == False 不需要metacell处理<br>
从filepath中读取BRCA_exp_filter_saver,(基因个数,细胞个数)<br>
2.读取KEGG<br>
3.Regnetwork_path = 'pathway/Regnetwork/2022.human.source'<br>
这是写死在函数里,不是传递的参数<br>
从这个路径获得human_network<br>
4.network_percent是一个DataFrame,<br>
列名为Pathway,NUM_ORIG,NUM_PCC,NUM_MI<br>
5.1 如果test == True,测试集<br>
通过from_cancer_create,matrix2Data两个函数获得batch<br>
return batch<br>
5.2 如果test == False,训练集<br>
pathway_ID_list是KEGG的键<br>
5.2.1 遍历pathway_ID_list中的Other_Pathway<br>
network_percent保存每条边的出节点和入节点<br>
data_list保存每个Data,即图<br>
edge_percent保存每个图中边占图的大小的比例,即稀疏程度<br>
5.2.2 return_list(默认为False)控制返回的data_list是list类型还是转换为tensor类型的batch.<br>
默认返回tensor类型的batch,以及edge_percent取平均值<br>
return batch, edge_percent

>复现<br>
先验信息的来源:KEGG,Regnetwork<br>
discrete文件夹下添加__init__.py文件,才能将该文件夹识别为模块,运行当前文件



- `pathway/simulation/`: synthetic network and gene expression profile generated by SERGIO
- `discrete/`: model-related configuration files
### 2.1 discrete/network_preprocess.py
>class PlaceHolder:<br>
1.def __init__(self, X, E, y):<br>
初始化<br>
2.def type_as(self, x: torch.Tensor):<br>
将X,E,y的类型修改为和x相同,即张量<br>
3.def mask(self, node_mask, collapse=False):<br>
node_mask:(bs,n)<br>
x_mask,e_mask1,e_mask2,都是在node_mask的基础上拓展维度,维度分别是:
(bs,n,1),(bs,n,1,1),(bs,1,n,1)<br>
3.1 如果collapse == True:<br>
更新E的形状为(bs, n, n)然后根据mask把含0的(被屏蔽)节点置-1<br>
3.2 如果collapse == False:<br>
更新X,把mask中0的点置0<br>
更新E,把mask中0的点置0<br>
return self

>def to_dense(x, edge_index, edge_attr, batch=None, training=True, max_num_nodes=None):<br>
x:(num_nodes, node_attr)<br>
batch:(num_nodes),用于指示x中的每一行(即每个节点)属于哪一个batch<br>
1.处理节点<br>
1.1 if training:<br>
X:(bs, max_num_nodes, node_attr)经过密集处理的批次张量<br>
node_mask:(bs, max_num_nodes)为0的节点指示为掩码,因为每个batch可能节点数量不同<br>
1.2 else:<br>
X = x<br>
node_mask = None<br>
2.处理边<br>
E: (bs, max_num_nodes, max_num_nodes),源代码中每条边的特征为常数,即权重.如果边的特征是向量,则形状为(bs, max_num_nodes, max_num_nodes,edge_attr)<br>
E_onehot:(s, max_num_nodes, max_num_nodes,2)独热编码,即最后一个维度是[1,0],(对应权重为0)或者[0,1],(对应权重为1)<br>
return PlaceHolder(X=X, E=E_onehot, y=None), node_mask,max_num_nodes

>def get_max_node(data):<br>
获得最大节点数,似乎把data是PlaceHolder类型的,还把X写成小写,而且其他地方没有用到,忽略

>def encode_no_edge(E):<br>
E:(bs, n, n, 2)<br>
然后写了个冗余的判断,跳过<br>
no_edge:(bs,n,n),bool类型,E中最后一个维度的和为0就置1<br>
first_elt:(bs,n,n),E的最后一个维度的第1维<br>
将这个first_elt中no_edge的位置置1,相当于没有边,再赋值给E的最后一个维度的第1维,即在矩阵中编码为[0,1]<br>
这又是一个错误,没有边应该编码为[1,0],应该在first_elt时取第0维而不是第1维<br>
diag:(bs,n,n)bool类型的对角阵<br>
然后用diag将E的对角线元素置0<br>
return E

>def unnormalize(X, E, y, norm_values, norm_biases, node_mask, collapse=False):<br>
对X,E,y进行反归一化,即乘标准差后加均值
return PlaceHolder(X=X, E=E, y=y).mask(node_mask, collapse)

>def normalize(X, E, y, norm_values, norm_biases, node_mask):<br>
对E,y归一化<br>
diag:(E.shape[0], E.shape[1], E.shape[1]),其中第1,2维的数组是对角线矩阵,类型为bool<br>
将E在diag为True的对应位置(对角线)置0<br>
return PlaceHolder(X=X, E=E, y=y).mask(node_mask)



### 2.2 discrete/diffusion_utils.py
>class CrossEntropyMetric(Metric):<br>
与discrete/models/train_metrics.py中同名的类相同,用于计算交叉熵

>class SumExceptBatchMetric(Metric):<br>
总和值/样本数,用于算出一个平均值

>class SumExceptBatchKL(Metric):<br>
用于计算p,q的kl散度的平均值

>class NLL(Metric):<br>
用于计算batch_nll的平均值

>def linear_beta_schedule(timesteps):<br>
用于生成 DDPM（Denoising Diffusion Probabilistic Models）中的线性噪声调度（beta schedule）。噪声调度控制着在扩散过程中如何逐步向数据添加噪声，以及在逆过程中如何逐步去噪。<br>
返回一个长度为timesteps的等差数列

>def cosine_beta_schedule_discrete(timesteps, s=0.008):<br>
生成余弦噪声调度，通过余弦函数平滑控制噪声强度<br>
返回长度为timesteps+1的噪声,随着时间步增加噪声逐渐增大

>def get_diffusion_betas(spec):<br>
得到噪声beta,既然有这个函数,上面两个函数基本就没用了<br>
spec:dict,其中可以指定噪声类型,有三种类型:linear,cosine,jsd<br>
jsd实际上是倒数调度,即1/T,1/(T-1),...1,这与jsd不同.以及他提供的论文DPM使用的二项式扩散过程的调度不同

>def custom_beta_schedule_discreteDig(timesteps, average_num_nodes=50, s=0.008):<br>
与余弦调度重复

>def sample_discrete_max(X, probE, node_mask, test=False):<br>
X: bs, n, dx_out<br>
probE: bs, n, n, de_out<br>
node_mask: bs, n<br>
inverse_edge_mask: bs, n, n,根据node_mask获得途中要屏蔽的边并取反, 结果上True表示要屏蔽,False表示不屏蔽<br>
diag_mask:bs, n, n,对角线为1的矩阵<br>
probE的对角线以及要屏蔽的位置,都置为1/ de_out<br>
E_t:(bs, n, n) 元素为probE最后一个维度取最大值的索引,然后把对角线置1
return PlaceHolder(X=X, E=E_t, y=None)

>def sample_discrete_features(X, probE, node_mask, randomseed=42, test=True):<br>
X: bs, n, dx_out<br>
probE: bs, n, n, de_out<br>
node_mask: bs, n<br>
inverse_edge_mask: bs, n, n,根据node_mask获得途中要屏蔽的边并取反, 结果上True表示要屏蔽,False表示不屏蔽<br>
diag_mask:bs, n, n,对角线为1的矩阵<br>
probE的对角线以及要屏蔽的位置,都置为1/ de_out<br>
probE形状改为(bs*n*n,de_out),最后一个维度即为抽样概率<br>
抽样后复原形状得到E_t:(bs,n,n),对角线置1<br>
return PlaceHolder(X=X, E=E_t, y=None)

>def Evaluation(y_true, y_pred, flag=False):<br>
y_p,从y_pred转化而来的np数组(n,)<br>
y_t,从y_true转化而来的np数组(n,)<br>
用y_t,y_p计算AUC<br>
用y_t,y_p计算roc曲线,包括:fpr, tpr, thresholds<br>
best_threshold = thresholds中令tpr - fpr最大的值<br>
用y_t,y_p计算PR曲线,包括:precision,recall,thresholds<br>
AUPR = PR曲线下面积<br>
AUPR_norm = AUPR / y_t均值(正样本比例)<br>
y_p对于大于best_thresholds的部分置1,其余部分置0,从概率分布变成0-1<br>
用y_t, y_p计算f1<br>
return {'AUC': AUC, 'AUPR': AUPR, 'AUPR_norm': AUPR_norm, 'F1': f1}

>def compute_batched_over0_posterior_distribution(X_t, Qt, Qsb, Qtb):<br>
对边计算后验分布<br>
0,t都是时间步,已知x_t,计算x_0的后验分布的公式:<br>
$$
\text{posterior} = \frac{(X_t \cdot Q_t^T) \odot (x_0 \cdot Q_{sb})}{x_0 \cdot Q_{tb} \cdot X_t^T}
$$
用贝叶斯公式理解,上面的s为t-1时刻:
$$
\begin{aligned}
p(x_{t-1}|x_{t},x_{0}) &= p(x_{t}|x_{t-1},x_{0})p(x_{t-1}|x_{0})/ p(x_{t}|x_{0})\\
&= p(x_{t}|x_{t-1})p(x_{t-1}|x_{0})/ p(x_{t}|x_{0})
\end{aligned}
$$

out输出的形状:(bs, n*n, d0, d_t - 1)<br>
其中,n是边数,d0,d_t是0或者t时刻的特征维度

>def sample_discrete_feature_noise(X, limit_dist, node_mask, seed=42):<br>
X:()<br>
node_mask:(bs, n_max)<br>
limit_dist:dict<br>
e_limit:(bs,n_max,n_max,de)边的分布概率<br>
U_E:(bs,n_max,n_max)每条边进行多元分布的采样结果,对角线置1<br>
long_mask:将node_mask改为long类型<br>
U_E修改为one-hot编码:(bs,n_max,n_max,de)<br>
y:(1,0)的空张量<br>
return PlaceHolder(X=X, E=U_E, y=y).mask(node_mask)

>def delte_dig_from_batch(batchdata):<br>
将batchdata的对角线置1然后返回batachdata

>def cal_del_TF_edge(GeneName):<br>
GeneName是一个包含了调控基因TF和靶基因Gene的list<br>
通过查表GRN/TF.txt,将这两者分开,得到他们在GeneName中的索引<br>
return GENE_ID_list, TF_ID_list

### 2.3 discrete/noise_predefined.py
>class PredefinedNoiseScheduleDiscrete(torch.nn.Module):
一个噪声调度的查找数组
1.初始化:
betas:噪声调度,不需要梯度更新的张量,有cosine和线性调度两个选择,diffusion_utils.get_diffusion_betas中的jsd调度没有使用到
alphas = 1 - betas
alphas_bar: alphas的累积乘积
2.def forward(self, t_normalized=None, t_int=None):
t_normalized,t_int中有且只有一个是None
t_normalized:取值0-1的浮点数,时间中的相对位置
t_int:整数,时间索引,如果为None,则将t_normalized乘以总步长(timesteps),然后四舍五入取整数
返回betas中t_int索引的值
return self.betas[t_int.long()]
3.def get_alpha_bar(self, t_normalized=None, t_int=None):
逻辑与上面相同
返回alphas_bar中t_int索引的值
return self.alphas_bar[t_int.long()]

>class DiscreteUniformTransition:
1.初始化
E_classes:状态的数量,没有显式的写为2
u_e: (1,E_classes,E_classes), 元素全为1/E_classes的矩阵
2.def get_Qt(self, beta_t, device):
beta_t:(bs,1)
q_e是从t-1时刻到t时刻的状态转移矩阵,随着时间t增大,I(原图)的影响变小,u_e(噪声)的影响增大
return network_preprocess.PlaceHolder(X=None, E=q_e, y=None)
3.def get_Qt_bar(self, alpha_bar_t, device):
alpha_bar_t: (bs,1)
q_e是从0时刻到t时刻的状态转移矩阵,随着时间t增大,I(原图)的影响变小,u_e(噪声)的影响增大
return network_preprocess.PlaceHolder(X=None, E=q_e, y=None)

>class MarginalUniformTransition:
1.初始化
e_marginals:长度为E_classes的张量,状态的数量没有显式的写为2
u_e:(1,E_classes,E_classes),元素是由参数传入的,与上面的均匀分布不同
2.def get_Qt(self, beta_t, device):与上面的类相同
3.def get_Qt_bar(self, alpha_bar_t, device):与上面的类相同


- `discrete/models/`: Graph Transformer architecture
>该目录下的graph_transformer.py使用dgl包.图神经网络常用的两个包pyg和dgl,本教程配置环境下载的是pyg,没有dgl,所以这个文件是无法直接运行的.<br>
事实上,本文使用的模型都在transformer_model.py文件内,名字和用法不是很匹配.
### 2.4 discrete/models/layers.py
>class Xtoy(nn.Module):<br>
处理节点特征,将平均值,最大值,最小值,标准差的信息结合起来然后通过一个线性层
这种特征处理方法叫PNA<br>
输入X: (bs, n, dx)<br>
输出out: (bs, dy)

>class Etoy(nn.Module):<br>
处理边的特征,将平均值,最大值,最小值,标准差的信息结合起来然后通过一个线性层<br>
输入E: (bs, n, n, de)<br>
输出out: (bs, dy)

>def masked_softmax(x, mask, **kwargs):<br>
掩码mask为0的部分忽略,但是这里似乎有误,如果mask全0,应该对整个张量都忽略(即返回全0张量), 但是原代码会返回原张量<br>
掩码后经过一个softmax<br>
返回的形状与x相同



### 2.5 discrete/models/train_matrics.py
>class CrossEntropyMetric(Metric):<br>
Metric 是一个抽象基类,用于定义和计算各种评估指标.<br>
preds和target:(bs * n, d) or (bs * n * n, d)<br>
total_ce 累积交叉熵<br>
total_samples 累积样本数量<br>
输出: total_ce/total_samples

>class NodeMSE(MeanSquaredError),class EdgeMSE(MeanSquaredError):<br>
分别是点和边的MSE.MSE用于回归任务,但是对于边似乎是0-1的分类问题,需要进一步分析

>class TrainLossDiscrete(nn.Module):<br>
1.初始化<br>
node_loss,edge_loss,y_loss均为交叉熵损失函数<br>
lambda_train是传入参数<br>
2.forward(self, masked_pred_E, nosoft_pred_E, true_E, CEloss=False):<br>
前向传播过程<br>
如果CEloss == True, masked_pred_E赋值为nosoft_pred_E
输入--masked_pred_E,true_E:(bs, n, n, de)<br>
reshape后--masked_pred_E,true_E:(bs * n * n, de)<br>
mask_E:true_E最后一个维度存在不为0的则为True,全0则False,(bs * n * n)<br>
masked_pred_E,true_E分别使用mask_E,删掉全为0的行,得到flat_pred_E,flat_true_E:(-1,de)<br>
如果CEloss为True,计算flat_pred_E,flat_true_E的交叉熵,否则计算他们的二元交叉熵,得到loss_train<br>
return loss_train<br>
3.reset(self):<br>
每次重置node_loss,edge_loss,y_loss<br>
评价:实际计算损失函数用到的是torch库自带的交叉熵和二元交叉熵,初始化定义的node_loss,edge_loss,y_loss没有使用到,意义不明



>def weights_init(m):<br>
m是神经网络中的层,用xavier初始化attention层的权重,用kaiming初始化其他层(如线性层)的权重,bias初始化为0.关于两种方法见`5.2初始化与激活函数`



### 2.6 discrete/models/trasformer_model.py
>class NodeEdgeBlock(nn.Module):<br>
自注意力机制的模块<br>
前向传播def forward(self, X, E, y, node_mask):<br>
X:(bs, n, d)点<br>
E:(bs,n,n,d)边<br>
y:(bs,dz)全局特征<br>
node_mask:(bs, n)<br>
Q,K,V:来自点的特征X<br>
Y:通过FiLM技术将边E的信息融合到点X(Q*K)中,关于FiLM技术,具体见5.3节,注意区分中间变量Y和全局特征y<br>
newE:通过FiLM技术将全局y的信息融合到Y中<br>
attn: Y经过掩码和softmax后,矩阵每一列的和为1<br>
weighted_V:将attn作为权重和V相乘,之后按列求和(bs, n, trans_GE)<br>
newX:通过FiLM技术将全局y的信息融合到点X(weighted_V)中<br>
new_y:X,E,y的信息经过线性层后直接相加<br>
return newX, newE, new_y<br>
返回形状与输入相同

>class Transformer_encoder(nn.Module):<br>
前向传播def forward(self, X, E, t, node_mask):<br>
self.self_attn = NodeEdgeBlock(trans_GE, trans_E, trans_T, n_head, **kw)即上面定义的自注意力模块,得到newX, newE, new_y<br>
对点,边,全局信息分别残差连接(如X + newX)后通过LayerNorm,更新X,E,t<br>
再次残差连接,这次是加上线性层(X + self.MLP_GE(X))后通过LayerNorm,更新X,E,t<br>
return X, E, t

>class DigNetGraphTransformer(nn.Module):<br>
前向传播def forward(self, x, edge_attr, t_step, edge_index, ALLedge_index, node_mask, noisy_data):<br>
这么多参数中,只有node_mask,noisy_data实际使用到<br>
noisy_data是一个字典,从中获得X,E,y<br>
diag_mask:(bs,n,n,1)掩码,对角线为False,其余为True<br>
after_in:对X,E,y进行MLP和掩码,是一个PlaceHolder类型数据,用它更新X,E,y<br>
然后经过多层(层数在初始化时定义)的Transformer_encoder,更新X,E,y<br>
输出层还是经过MLP,更新X,E,y<br>
E经过diag_mask掩码, 对角线置0<br>
return network_preprocess.PlaceHolder(X=X, E=E, y=t).mask(node_mask)

In [1]:
import torch
from torch_geometric import utils
from torch_geometric.data import Data,Batch
a = torch.rand(3,6,9)
#utils.dense_to_sparse(torch.ones(3,3))[0].shape
x = [[1],[2],[3]]
e = [[1,2],[0.3,0.7],[0.4,0.6]]
d = Data(x = x, edge_attr = e)
b = torch.zeros(3,4,5)
b[1,:,:] = 1
#b.nonzero(as_tuple = True)
#b.nonzero().t().shape
torch.tensor([[1, 0]], dtype=torch.float32).repeat(3,2)

tensor([[1., 0., 1., 0.],
        [1., 0., 1., 0.],
        [1., 0., 1., 0.]])


- `denoising_diffusion_pytorch/`: Contains related sub-function files used for training/testing/initializing DigNet
该目录下只有一个文件denoising_diffusion_pytorch_1d.py
### 2.7 denoising_diffusion_pytorch/denoising_diffusion_pytorch_1d.py
>class RowNormalize(nn.Module):<br>
对每行用L2范数进行归一化

class GaussianDiffusion1D(nn.Module):
>初始化def __init__(
        self,
        model,
        *,
        edge_percent=0.3,
        max_num_nodes=None,
        device='cpu',
        timesteps = 1000,
        sampling_timesteps = None,
        loss_type = 'l2',
        predflag = 'E_start',
        transition='marginal',  # 转移矩阵类型
        ddim_sampling_eta=0.,
        noise_type='cos',
    ):<br>

>def noisy_data2data_dense(self, noisy_data, all_edge=True):
noisy_data:dict,索引有X_t,E_t<br>
data_list_ALL,data_list,两个list用于在接下来的循环中保存data_ALL和data<br>
1.循环遍历bs<br>
从noisy_data中获得边和点的切片<br>
edge_weights:(n,n,de)邻接矩阵<br>
node_features:(n,dx)<br>
edge_index:(2,n * n),所有边(不管存不存在)的索引,比如第一列为[0,0],第二列为[0,1]...<br>
data_ALL:pyg中的Data类,包括x,edge_attr,edge_index,包括所有边(n*n条),不管权重是否为0<br>
edge_weights0 = edge_weights[:, :, 0]<br>
indices_tensor:(2,x):x是非零的边的数量,找到edge_weight0中所有不为0的边的索引<br>
num_edges:不为0的边的数量<br>
values_tensor :(num_edges,2),每一行都为[1,0]的张量<br>
data:pyg中的Data类,包括x,edge_attr,edge_index,只包括权重不为0的边<br>
把data_ALL,data添加到data_list_ALL,data_list中<br>
2.<br>
batch_ALL:data_list_ALL转换为batch数据<br>
batch:data_list转换为batch数据<br>
ALLedge_index:(n * n * bs, 2)整个batch_ALL的索引<br>
3.<br>
如果all_edge为True(默认情况),return batch_ALL, ALLedge_index<br>
否则return batch, ALLedge_index<br>

>def pred_shape2data(self, pred, noisy_data):<br>
pred的形状与noisy_data中的E_t一致:(bs,n,n,c)<br>
pred可能是张量(只包含边)或者PlaceHolder(包含点,边,全局),无论是哪种,都返回PlaceHolder<br>
return network_preprocess.PlaceHolder(X=pred.X, E=pred.E, y=pred.y)<br>

>def loss_fn(self):<br>
l1或者l2损失函数,初始化时默认l2损失函数<br>

>def compute_posterior_distribution(self, M, M_t, Qt_M, Qsb_M, Qtb_M):<br>
计算后验分布,用于扩散模型的反向过程, 即去噪过程<br>
M:(bs, N = n*n, d)在下面调用时输入是E,即边的信息<br>
M_t:(bs, n*n, d)<br>
Qt_M:(bs,d,d)<br>
Qsb_M:(bs,d,d)<br>
Qt_M_T:(bs,d,d)转置<br>
return prob<br>
返回后验概率的形状:(bs,N,d)

>def posterior_distributions(self, E, E_t, Qt, Qsb, Qtb):<br>
用compute_posterior_distribution函数计算边的后验概率prob_E<br>
return network_preprocess.PlaceHolder(X=1, E=prob_E, y=None)

>def compute_Lt(self, E, pred, noisy_data, node_mask, test=False):<br>
pred包含边的数据,noisy_data包含点,边,全局信息<br>
pred_probs_E:PlaceHolder类型,点,全局信息来自noisy_data,边来自pred<br>
pred_probs_E0:pred_probs_E的副本<br>
Qtb:PlaceHolder类型,从noisy_data['alpha_t_bar']获得0到t时刻的alpha_bar<br>
Qsb:PlaceHolder类型,从noisy_data['alpha_s_bar']获得0到s时刻的alpha_bar<br>
Qt:PlaceHolder类型,从noisy_data['beta_t']获得t-1到t时刻的alpha<br>
prob_true:PlaceHolder,用E(干净数据)计算后验概率<br>
prob_pred:PlaceHolder,用 pred_probs_E 计算后验概率<br>
prob_true_E, prob_pred.E 经过掩码的先验和后验分布
这两者计算KL散度,得到kl_e
return kl_e, prob_true_E

>def vb_terms_bpd(self, E, pred, noisy_data, node_mask):<br>
代表变分下界(Variational Lower Bound)，即 ELBO(Evidence Lower Bound).在变分自编码器(VAE)等模型中,VB 是优化目标,用于近似复杂的后验分布.<br>
pred包含边的数据,noisy_data包含点,边,全局信息<br>
pred_probs_E:(bs, n, n, de)来自pred的信息<br>
pred_probs_E0:pred_probs_E的副本<br>
Qtb:PlaceHolder类型,从noisy_data['alpha_t_bar']获得0到t时刻的alpha_bar<br>
Qsb:PlaceHolder类型,从noisy_data['alpha_s_bar']获得0到t时刻的alpha_bar<br>
Qt:PlaceHolder类型,从noisy_data['beta_t']获得t-1到t时刻的alpha<br>
prob_true:PlaceHolder,用E(干净数据)计算后验概率<br>
prob_pred_t_1:(bs,n*n,d0,d_t-1) 从0到t的后验<br>
pred_probs_E 修改形状为(bs, n*n, de)<br>
weighted_E:(bs,n*n,d0,d_t-1)以上两项的哈达马积<br>
unnormalized_prob_E:(bs, n*n,d_t-1),对weighted_E倒数第二个维度求和,为0的元素改为1e-5<br>
prob_E_final:由unnormalized_prob_E得到,最后一个维度和为1,形状改为(bs,n,n,d_t-1)<br>
t0mask:(t, 1),bool类型,如果t为0则True,其他False,即时间步t为0的索引<br>
对于t0mask的地方(时间步为0的地方), prob_E_final 用pred_probs_E0 覆盖<br>
prob_true.E 用 E 覆盖<br>
prob_true_E, prob_E_final = self.mask_distributions(true_E=prob_true.E, pred_E=prob_E_final, node_mask=node_mask)<br>
loss = self.train_loss(masked_pred_E=prob_E_final, nosoft_pred_E=prob_E_final, true_E=prob_true_E)<br>
return loss, prob_E_final, prob_true_E

>def loss_xtpxt(self, E, pred, noisy_data, node_mask):<br>
vb和ce损失结合得到losses<br>
return losses

>def reconstruction_logp(self, X, t, E, node_mask):<br>
t:tensor,形状未知<br>
beta_0:(bs)从噪声调度中选择t = 0 的值<br>
Q0:PlaceHolder,获得0到1时刻的状态转移矩阵<br>
probE0:(bs,n,n,de_out)<br>
sampled0:PplaceHolder,通过probE0采样,边经过采样的整数,形状是(bs,n,n)<br>
E0:sampled0经过one-hot编码,(bs,n,n,de_out)<br>
noisy_data = {'X_t': X, 'E_t': sampled_0.E, 'y_t': None, 'node_mask': node_mask,'t_int': torch.zeros(X.shape[0], 1).type_as(E)}<br>
用noisy_data获得batch, ALLedge_index<br>
time_tensor_tonode:(bs * n)每个元素都是0<br>
pred = 通过model得到的预测值<br>
probE0:(bs,n,n,de_out),对pred的最后一个维度softmax<br>
然后对于矩阵中被掩码的部分,将最后一个维度全置为1(作为均匀分布,为什么不是1/de_out)<br>
return network_preprocess.PlaceHolder(X=X, E=probE0, y=None)<br>

>def p_losses(self, E, noisy_data, node_mask, loss_method='celoss', calAUC=True):<br>
评价:默认的损失函数是交叉熵celoss,没有使用变分下界. 默认calAUC = True,
另一种不计算AUC的方法,输出个数不同,可能导致报错,没有被使用.
计算E与pred的二元交叉熵,相当于直接比较预测值,
与扩散模型中常见的比较噪声的MSE不同.<br>
调用上面的函数noisy_data2data_dense(noisy_data, all_edge=False),得到batch, ALLedge_index.
E 是干净的原始数据,noisy_data 是加噪声数据<br>
batch不含权重为0的边, ALLedge_index则包含所有边的所有索引<br>
time_tensor_tonode:(bs*n,?, 1)noisy_data中的t(即全局信息),按batch的索引进行选择,每个batch中有n个点,所以第一个维度是bs * n<br>
pred = self.model(batch.x, batch.edge_attr, time_tensor_tonode, batch.edge_index, ALLedge_index, node_mask, n)<br>
这个类中没有默认model的值,不过根据传参基本可以确定模型是
discrete/models/transformer_model.DigNetGraphTransformer,
最后返回的是PlaceHolder类型数据pred<br>
mask_pred_probs_E:pred经过掩码<br>
mask_pred_probs_ES:pred边的数据,对最后一个维度softmax<br>
predflag如果是'E_start'(默认值),target = E,即对原始干净的边预测<br>
否则target = noisy_data['E_s']<br>
loss:对于变分下界(vb)的损失函数或者交叉熵损失函数<br>
1.1  如果calAUC为True<br>
E_pred_discrete:(bs,n,n)最后一个维度已经argmax<br>
E_true_discrete:(bs,n,n)最后一个维度已经argmax<br>
AUC_list:对每个bs保存AUC<br>
AUC:AUC_list中的最小值<br>
return loss, pred, AUC<br>
1.2 如果calAUC为False<br>
return loss, pred

>def given_pred_to_edge(self, pred, noisy_data, node_mask, predflah2 = 'noweight'):<br>
根据后验概率pred采样获得边的预测结果<br>
X_t:(bs,n,dx)<br>
beta_t:(bs,1)<br>
alpha_s_bar:(bs,1)<br>
alpha_t_bar:(bs,1)<br>
状态转移矩阵<br>
Qtb:(bs,E_classes,E_classes)<br>
Qsb:(bs,E_classes,E_classes)<br>
Qt:(bs,E_classes,E_classes)<br>
pred_probs_E:(bs,n,n,d0) 经过采样,最后一个维度one-hot,
对角线和mask全为0.他想要的应该是一个概率,为何要one-hot?<br>
prob_E_final:(bs,n,n,d_t-1)后验概率分布<br>
E_s:(bs,n,n,Edim_output)根据prob_E_final采样的结果,最后一个维度是one-hot<br>
return E_s

>def apply_noise(self, X, E, y, node_mask,t_int=None):<br>
t_int:(bs,1)<br>
s_int = t_int - 1, 即s是t的前一时刻,如果t = 0则s = 0<br>
t_float:(bs,1),元素为0到1的浮点数,整数时间步占总步长的比例<br>
s_float:(bs,1),元素为0到1的浮点数,整数时间步占总步长的比例<br>
beta_t,alpha_s_bar,alpha_t_bar:(bs,1),噪音调度<br>
Qtb:(bs,E_classes,E_classes)0到t时刻的转移矩阵<br>
probE:(bs,n,n,de_out)t时刻的概率分布<br>
E_t:(bs,n,n,de_out)t时刻的抽样结果,最后一维one-hot<br>
z_t:PlaceHolder<br>
对s时刻同样操作得到E_s,z_s<br>
将t和s时刻的信息(主要包括时间,beta,alpha,边)放入dict类型的noisy_data中<br>
return noisy_data

>def mask_distributions(self, true_E, pred_E, node_mask):<br>
pred_E:(bs,n,n,de) 把0变成小值,防止数值问题<br>
row_E:(de)显式把第1个索引设置成1,即[0,1]<br>
true_E,pred_E的最后一个维度都变成[0,1],那刚才对pred_E的处理不是没用了?<br>
return true_E, pred_E<br>

>def kl_prior(self, X, E, node_mask):<br>
据注释,后验 q(z1 | x)和先验 p(z1)的kl散度,在实验中不那么重要,但是能帮助查看噪声调度是否正确.<br>
Ts:(bs,1) 全为 T - 1(总时间的前一时刻) <br>
Qtb:0到T-1时刻的状态转移矩阵<br>
probE:0到T-1时刻的分布矩阵<br>
limit_E :(bs,n,n,E_classes)类的初始化时定义的分布概率<br>
kl_distance_E:(bs,n,n,E_classes)初始化(先验)的limit_E和后验probE的kl散度<br>
return self.sum_except_batch(kl_distance_E)<br>
返回形状(bs),对第一个维度以外求和.

>def compute_val_loss(self, pred, noisy_data, X, E, node_mask, test=False):<br>
变分下界<br>
N:(bs)<br>
log_pN:(bs)每个batch的节点数量与最大节点数的比值,取对数<br>
kl_prior:(bs ) 先验E和加噪声后的分布的kl散度<br>
loss_all_t:标量, 扩散的损失, 计算仍然使用kl散度<br>
prob0:PlaceHolder通过模型对0时刻的分布进行预测<br>
loss_term_0: 0时刻真实值和预测值的交叉熵<br>
nlls = -log_pN + kl_prior + loss_all_t - loss_term_0<br>
因为上面的对数都是负数,取负号为正.对batch维度求和<br>
return nll

>def forward(self, data, *args, **kwargs):<br>
输入data是Batch类型<br>
转换为PlaceHolder类型的dense_data
从中获得X,E<br>
y是形状(num_graphs,0)的空矩阵, 这里应该有修改的空间<br>
noisy_data:dict类型,包含t和t-1时刻的加噪数据<br>
loss, pred, AUC计算损失函数<br>
return loss, AUC

>def validation_step(self, data):<br>
在不计算梯度的情况下,计算多种损失的总和,<br>
return val_loss

>def sample_zt(self, noisy_data):<br>
获得t时刻的采样值<br>
pred1:t时刻的预测概率<br>
E_s1:采样值<br>
out_one_hot,out_discrete两者一样,包括了边的采样值<br>
return out_one_hot.mask(node_mask).type_as(y_t), out_discrete.mask(node_mask, collapse=False).type_as(y_t)

>def sample_p_zs_given_zt(self, s, t, X_t, E_t, y_t, node_mask):<br>
t到t-1时刻的采样值<br>
noisy_data:仅含t时刻的数据,不含s(t-1)时刻<br>
pred:<br>
dense2data:该函数不存在,说明这个函数可能有问题,功能与sample_zt几乎相同

>sample_zt_for_photo(self, noisy_data):<br>
通过模型获得t-1的预测概率pred1,然后采样得到E_s1<br>
E_t:(1,51,51,1)预测概率<br>
Et_mask:(1,51,51,1) 预测的边<br>
out_one_hot,out_discrete:PlaceHolder,都是预测的边<br>
photo_data:包含预测概率和预测值<br>
return out_one_hot.mask(node_mask).type_as(y_t), out_discrete.mask(node_mask, collapse=False).type_as(y_t), photo_data

>def test_step(self, testdata, TrueData=None, show=True, seed=None):<br>
testdata只包含一个批次,即bs = 1, 没有bs维度<br>
X:(n, dx) n为节点个数<br>
dense_X:(1,n_max,dx) 其中n_max (>= n) 是所有批次中最大节点数 <br>
node_mask:(1, n_max) 掩码,前n个为True, 后面False<br>
z_T:PlaceHolder, 随机采样的结果<br>
E,y: z_T中的边和全局信息<br>
pbar2:进度条,T-1 到 0 <br>
1.对pbar2中的t_int循环:<br>
t_array:(1,1)递减的t时刻 <br>
s_array:(1,1)递减的t-1时刻,t == 0时和t相等<br>
noisy_data:dict,包含t,s时刻的边<br>
sampled_s:PlaceHolder,采样值<br>
1.1 默认show == True,展示:t, AUC, AUPR, AUPRM<br>
all_adj中加入预测值的邻接矩阵<br>
2.return sampled_s1.E, all_adj<br>
返回最后一个(0时刻)的预测值, 以及所有时刻的预测值





- `config.py`: Configure hyperparameters for training or testing DigNet
- `DigNet.py`: Contains the process framework of DigNet
主程序

2.8 Dignet.py/class Dignet
>def __init__(self, args):<br>
初始化的args是从config/config.json下读取

>def load_pre_model(self, diffusion_pre):<br>
diffusion_pre:dict,含有模型参数和预训练参数<br>
从diffusion_pre中读取参数<br>
model:DigNetGraphTransformer<br>
model 加载预训练参数<br>
diffusion:GaussianDiffusion1D<br>
return diffusion

>def load_pretrained_model(self, diffusion_pre):<br>
与load_pre_model相同

>def evaluate(self, testdata, truelabel):<br>
主要调用GaussianDiffusion1D.test_step<br>
return all_adj

>def setup_paths(self):<br>
如果不存在,创建results_checkpoint/TrainingData文件<br>

>def load_test_data(self, test_filename, num, diffusion_pre):<br>
获得test_data:Batch<br>
truelabel:PlaceHolder<br>
trulabel_discrete:(n,n,de)
return testdata, truelabel_discrete

>def load_data(self, train_filename, n_train=200, n_test=[1000, 1010]):<br>
data:Batch<br>
edge_percent<br>
test_list:每个元素是字典,包含一个测试的图的Data和边<br>
train_file = 'result/Training data_BATCH.data'<br>
data:Batch,训练数据<br>
更新self.data,self.edge_percent,self.test_list

>def setup_model(self):<br>
设置模型,需要在加载数据之后
将加载数据得到的data转换为dataloader
train_node:batch中节点数的最大值
学习率调度:每100步变为0.95倍

>def train(self, train_filename, n_train=200, n_test=[1000, 1010]):
加载数据,设置模型
1. 遍历epoch:
1.1 遍历graph_data_loader:
通过扩散模型计算loss, Train_AUC
1.2 更新学习率;计算平均值total_loss,total_AUC
每10个epoch输出一次平均损失
每200个epoch保存一次中间模型checkpoint,results_checkpoint文件夹下
每100个epoch进行一次validation,其中计算best_mean_AUC,best_mean_AUC_file,printf,都是在验证集上
2. 遍历完所有epoch后保存模型,result文件夹下
return self.best_mean_AUC, self.best_mean_AUC_file, self.printf

>def test(self, diffusion_pre, testdata, truelabel=None):
加载训练过的模型
多进程进行训练,评估结果保存到all_adj_list = []
adj_final:
return adj_final

def evaluation(self, adj_final, truelabel):
输出diffusion_utils.Evaluation的结果performance
return performance

def save_model(self, filename):
用于保存checkpoint和最终模型

def validation(self, epoch):
在第epoch次,对验证集评估
输出AUC,AUPR,AUPRM,F1
如果AUC比以前更高,更新self.best_mean_AUC,self.best_mean_AUC_file

2.9 DigNet.py/main
1.1 输入表达谱和GRN

1.2 只输入表达谱

- `Download_TF_file.py`: used to download TF list
- `make_final_net.py`: Integrated voting sub-function
- `Tutorial.py`: A quick tutorial for using DigNet

## 3.复现代码

### 3.1 pathway/pathway.py

pathway/pathway.py中的主程序
功能:创建data类型和mat类型的数据,保存在Cancer_datasets文件夹下
有两种移除边的方式,1.基于先验知识(查表) 2.基于cmi
增加输出判断采用了哪种方式, 发现所有文件都使用了cmi,但是有以下两个问题
1. 计算cmi时,使用离散化而非连续化数据, 但是又假设是多元正态分布
2. 删除边的两种方式得到的边数用0.5进行比较而不是1,这样选边较少的图几乎完全会选第二种cmi方法

In [1]:
import scipy.io
from torch_geometric.data import Data, Batch
import numpy as np
import pickle
import torch
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import os
from tqdm import tqdm
import subprocess
from sklearn.decomposition import PCA
import scipy.io as sio
from sklearn.metrics import mutual_info_score
from sklearn.neighbors import NearestNeighbors
import random
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
from discrete.diffusion_utils import cal_del_TF_edge
from pathway.PCA_CMI import pca_cmi
from pathway.pathway import *
import powerlaw
from tqdm import tqdm

In [2]:
celllist = ["T_cell", "B_cell", "Cancer"]
filelist = [33,39,42,53,60]
# 此处根据修改Cancer_datasets中的文件修改,只做其中几个用于展示
test_pathway = 'hsa05224'
Cnum = 100
k = 20
KEGG = load_KEGG()
# KEGG是dict,每一个key对应多个基因名
Regnetwork_path = 'pathway/Regnetwork/2022.human.source'
dtypes = {1: str, 3: str}

#设置一个用于测试的文件路径
filepath = 'Cancer_Datasets/S33_Cancer_BRCA_output.csv'

# 第1,3列是数字,此处强行要求识别成字符串
human_network = pd.read_csv(Regnetwork_path, sep='\t', header=None, dtype=dtypes)
# human_network是DataFrame,共4列,第0,2列分别是调控基因TF,靶基因Gene
metacell = False

In [4]:
human_network.head()

,0,1,2,3
0,ALDOA,226,FBP1,2203
1,ALDOB,229,FBP1,2203
2,ALDOC,230,FBP1,2203
3,ALDH2,217,ADH1A,124
4,ALDH1B1,219,ADH1A,124


In [ ]:
# KEGG,dict,每一个key对应一个含若干基因的list
KEGG['hsa05224']

['ESR1',
 'ESR2',
 'NCOA1',
 'NCOA3',
 'FOS',
 'JUN',
 'SP1',
 'CCND1',
 'MYC',
 'PGR',
 'WNT1',
 'WNT4',
 'TNFSF11',
 'ERBB2',
 'FGF1',
 'FGF2',
 'FGF3',
 'FGF4',
 'FGF17',
 'FGF6',
 'FGF7',
 'FGF8',
 'FGF9',
 'FGF10',
 'FGF16',
 'FGF5',
 'FGF18',
 'FGF20',
 'FGF22',
 'FGF19',
 'FGF21',
 'FGF23',
 'FGFR1',
 'IGF1',
 'IGF1R',
 'EGF',
 'EGFR',
 'KIT',
 'SHC1',
 'SHC2',
 'SHC3',
 'SHC4',
 'GRB2',
 'SOS1',
 'SOS2',
 'HRAS',
 'KRAS',
 'NRAS',
 'ARAF',
 'BRAF',
 'RAF1',
 'MAP2K1',
 'MAP2K2',
 'MAPK1',
 'MAPK3',
 'PIK3CA',
 'PIK3CD',
 'PIK3CB',
 'PIK3R1',
 'PIK3R2',
 'PIK3R3',
 'PTEN',
 'AKT1',
 'AKT2',
 'AKT3',
 'MTOR',
 'RPS6KB1',
 'RPS6KB2',
 'JAG1',
 'JAG2',
 'DLL3',
 'DLL1',
 'DLL4',
 'NOTCH1',
 'NOTCH2',
 'NOTCH3',
 'NOTCH4',
 'HES1',
 'HES5',
 'HEYL',
 'HEY1',
 'HEY2',
 'FLT4',
 'CDKN1A',
 'NFKB2',
 'WNT2',
 'WNT2B',
 'WNT3',
 'WNT3A',
 'WNT5A',
 'WNT5B',
 'WNT6',
 'WNT7A',
 'WNT7B',
 'WNT8A',
 'WNT8B',
 'WNT9A',
 'WNT9B',
 'WNT10B',
 'WNT10A',
 'WNT11',
 'WNT16',
 'FZD1',
 'FZD7',
 '

In [3]:
BRCA_exp_filter_saver = pd.read_csv('Cancer_Datasets/S33_Cancer_BRCA_output.csv')

BRCA_exp_filter_saver.head()
# BRCA_exp_filter_saver的形状:(基因个数,细胞个数)

,Unnamed: 0,sc5rJUQ033_AAAGATGAGTTTCCTT,sc5rJUQ033_AAAGATGGTGGCCCTA,sc5rJUQ033_AAAGTAGTCCCTCAGT,sc5rJUQ033_AAATGCCGTCAGCTAT,sc5rJUQ033_AACACGTTCCGAAGAG,sc5rJUQ033_AACTTTCGTTCGGCAC,sc5rJUQ033_AAGACCTGTAAACACA,sc5rJUQ033_AAGGAGCAGTGAAGTT,sc5rJUQ033_AATCGGTTCTTGTTTG,...,sc5rJUQ033_TTATGCTTCTCTTATG,sc5rJUQ033_TTCCCAGGTGGGTCAA,sc5rJUQ033_TTCGGTCAGGGTGTGT,sc5rJUQ033_TTCGGTCTCATTCACT,sc5rJUQ033_TTCTACATCTGTTTGT,sc5rJUQ033_TTCTCAATCTTTACGT,sc5rJUQ033_TTTACTGCACCAGGTC,sc5rJUQ033_TTTATGCCACCTTGTC,sc5rJUQ033_TTTCCTCAGATCTGAA,sc5rJUQ033_TTTGGTTGTAAGAGAG
0,FO538757.2,0.953,1.420,1.441,1.465,0.716,0.466,0.777,0.490,1.198,...,1.682,0.823,0.788,1.228,2.305,1.485,1.119,0.996,2.284,1.469
1,RP11-206L10.9,0.188,0.246,0.189,0.188,0.179,0.193,0.299,0.187,0.192,...,0.194,0.190,0.195,0.239,0.196,0.191,0.183,0.198,0.191,0.188
2,NOC2L,0.522,0.428,0.426,0.422,0.384,0.441,0.422,0.415,0.540,...,0.548,0.427,0.449,0.408,0.455,0.434,0.496,0.461,0.533,0.419
3,PLEKHN1,0.133,0.140,0.137,0.441,0.103,0.154,0.133,0.127,0.148,...,1.234,0.139,0.164,0.121,0.171,0.146,0.380,0.180,0.469,0.131
4,HES4,1.554,1.122,3.421,1.696,0.783,0.997,1.927,1.395,0.720,...,3.388,1.720,0.974,0.737,3.660,1.437,1.215,0.865,0.918,1.065


In [4]:
BRCA_exp_filter_saver.set_index(BRCA_exp_filter_saver.columns[0], inplace=True)
#原代码多删了第一列,以下是正确的结果
BRCA_exp_filter_saver.head()

,sc5rJUQ033_AAAGATGAGTTTCCTT,sc5rJUQ033_AAAGATGGTGGCCCTA,sc5rJUQ033_AAAGTAGTCCCTCAGT,sc5rJUQ033_AAATGCCGTCAGCTAT,sc5rJUQ033_AACACGTTCCGAAGAG,sc5rJUQ033_AACTTTCGTTCGGCAC,sc5rJUQ033_AAGACCTGTAAACACA,sc5rJUQ033_AAGGAGCAGTGAAGTT,sc5rJUQ033_AATCGGTTCTTGTTTG,sc5rJUQ033_ACAGCTAAGGGTCGAT,...,sc5rJUQ033_TTATGCTTCTCTTATG,sc5rJUQ033_TTCCCAGGTGGGTCAA,sc5rJUQ033_TTCGGTCAGGGTGTGT,sc5rJUQ033_TTCGGTCTCATTCACT,sc5rJUQ033_TTCTACATCTGTTTGT,sc5rJUQ033_TTCTCAATCTTTACGT,sc5rJUQ033_TTTACTGCACCAGGTC,sc5rJUQ033_TTTATGCCACCTTGTC,sc5rJUQ033_TTTCCTCAGATCTGAA,sc5rJUQ033_TTTGGTTGTAAGAGAG
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
FO538757.2,0.953,1.420,1.441,1.465,0.716,0.466,0.777,0.490,1.198,0.945,...,1.682,0.823,0.788,1.228,2.305,1.485,1.119,0.996,2.284,1.469
RP11-206L10.9,0.188,0.246,0.189,0.188,0.179,0.193,0.299,0.187,0.192,0.246,...,0.194,0.190,0.195,0.239,0.196,0.191,0.183,0.198,0.191,0.188
NOC2L,0.522,0.428,0.426,0.422,0.384,0.441,0.422,0.415,0.540,0.531,...,0.548,0.427,0.449,0.408,0.455,0.434,0.496,0.461,0.533,0.419
PLEKHN1,0.133,0.140,0.137,0.441,0.103,0.154,0.133,0.127,0.148,0.141,...,1.234,0.139,0.164,0.121,0.171,0.146,0.380,0.180,0.469,0.131
HES4,1.554,1.122,3.421,1.696,0.783,0.997,1.927,1.395,0.720,1.337,...,3.388,1.720,0.974,0.737,3.660,1.437,1.215,0.865,0.918,1.065


In [ ]:
# 优化后的函数用时30s,比优化前(10min)快大约20倍
BRCA_exp_filter_saver_metacell = cal_metacell(BRCA_exp_filter_saver)
calRegnetwork1(human_network, BRCA_exp_filter_saver_metacell.index.to_list())

,TF,Gene
0,PDHA1,DLAT
1,PDHB,DLAT
2,DLAT,DLD
3,PKM,ENO1
4,PGAM1,PGK1
...,...,...
69566,ZRANB2,YY1
69567,ZRANB2,ZRANB2
69568,ZSCAN16,SLC25A38
69569,ZSCAN16,ZNF446


In [ ]:
#优化前, 结果相同,耗时较长
calRegnetwork(human_network, BRCA_exp_filter_saver_metacell.index.to_list())

,TF,Gene
0,ISG15,DDX58
1,ISG15,IFIH1
2,AGRN,DAG1
3,DVL1,GSK3B
4,DVL1,RAC1
...,...,...
69566,COL6A1,ITGB1
69567,COL6A1,CD44
69568,COL6A2,ITGB1
69569,COL6A2,CD44


In [11]:
def is_symmetric(matrix):
    # 检查是否为二维数组且行列数相等（方阵）
    if matrix.ndim != 2 or matrix.shape[0] != matrix.shape[1]:
        return False
    # 使用allclose处理浮点数精度问题
    return np.allclose(matrix, matrix.T)

a = np.array([[1,2],[2,1]])
is_symmetric(a)

True

In [ ]:
for si in celllist:
        for ji in filelist:
            # 39_Cancer作为测试集
            if not((str(ji) == '39') and (si == "Cancer")):
                #原代码路径有误,已根据实际情况修改
                filepath = 'Cancer_Datasets/S'+str(ji)+'_' + si+'_BRCA_output.csv'
                BRCA_exp_filter_saver = pd.read_csv(filepath)
                #原代码在删除第一列的操作有误,多删除了一列,已修改
                BRCA_exp_filter_saver.set_index(BRCA_exp_filter_saver.iloc[:,0],inplace=True)
                #BRCA_exp_filter_saver.drop(columns = BRCA_exp_filter_saver.columns[0],inplace = True)
                BRCA_exp_filter_saver_metacell = cal_metacell(BRCA_exp_filter_saver, Cnum=Cnum, k=k)# (基因个数,metacell个数)
                filepath = filepath.replace('output','output_meta')
                if BRCA_exp_filter_saver is not None:
                    parm = {'pear_percent': 4, 'MI_percent': 4, 'pmi_percent': 0.001}
                    [exp, adj_matrix, new_row] = from_cancer_create(BRCA_exp_filter_saver_metacell,
                                                                    KEGG,
                                                                    parm,
                                                                    test_pathway=None,
                                                                    Other_Pathway=test_pathway,
                                                                    human_network=human_network)
                    # 此处容易误会的是Other_Pathway=test_pathway,不是参数中的test_pathway=None,
                    # 而是在上方单元格定义的test_pathway.
                    # Other_Pathway决定了exp在KEGG中的哪个子集
                    """ data1 = np.sum(adj_matrix, axis=0)
                    fit1 = powerlaw.Fit(data1)
                    data2 = np.sum(adj_matrix, axis=1)
                    fit2 = powerlaw.Fit(data2)
                    data3 = data1+data2
                    fit3 = powerlaw.Fit(data3)
                    print("Alpha (exponent,0):", fit1.alpha, "Alpha (exponent,1):", fit2.alpha, "Alpha (exponent,sum):", fit3.alpha) """
                    #源代码中以上这段拟合意义不明,没有在其他地方出现,而且数据不符合幂律分布拟合的要求(全部大于0),所以注释掉
                    data = {"net": adj_matrix, "exp": np.array(exp), "genename": exp.index}
                    print(is_symmetric(adj_matrix))
                    print(new_row)
                    filename = filepath.replace("csv", "data")
                    f = open(filename, 'wb')
                    pickle.dump(data, f)
                    f.close()

                    sio.savemat(filepath.replace("csv", "mat"), {"net": adj_matrix, "exp": np.array(exp), "genename": exp.index})
                    print(filename+' is OK!!!!')

# 输出解释:
# 第一行:pearson-mi,cmi边的数量与比值
# 第二行:预处理选择了pearson-mi/cmi中的一个图
# 第三行:邻接矩阵是否对称
# 第四行:图的统计信息
# 第五行:文件夹处理完毕的输出

边的数量 169 166 1.0180722891566265
计算GRN的方法: cmi
False
{'Pathway': 'hsa05224', 'NUM_ORIG': 67.46987951807229, 'NUM_PCC': 18.92661555312158, 'NUM_MI': 13.603504928806133}
Cancer_Datasets/S33_T_cell_BRCA_output_meta.data is OK!!!!
边的数量 187 199 0.9396984924623115
计算GRN的方法: cmi
False
{'Pathway': 'hsa05224', 'NUM_ORIG': 85.42713567839196, 'NUM_PCC': 14.070351758793967, 'NUM_MI': 0.5025125628140702}
Cancer_Datasets/S39_T_cell_BRCA_output_meta.data is OK!!!!
边的数量 153 174 0.8793103448275862
计算GRN的方法: cmi
False
{'Pathway': 'hsa05224', 'NUM_ORIG': 80.45977011494253, 'NUM_PCC': 19.54022988505747, 'NUM_MI': 0}
Cancer_Datasets/S42_T_cell_BRCA_output_meta.data is OK!!!!
边的数量 283 309 0.9158576051779935
计算GRN的方法: cmi
False
{'Pathway': 'hsa05224', 'NUM_ORIG': 77.02265372168284, 'NUM_PCC': 11.974110032362459, 'NUM_MI': 11.003236245954692}
Cancer_Datasets/S53_T_cell_BRCA_output_meta.data is OK!!!!
边的数量 239 265 0.9018867924528302
计算GRN的方法: cmi
False
{'Pathway': 'hsa05224', 'NUM_ORIG': 77.35849056603774, 'NUM

此节以下部分不用运行

In [ ]:
# 测试数据,Data
testdata = create_batch_dataset_from_cancer(filepath=filepath,
                    test_pathway=test_pathway, test=True)
test_file = 'reproducibility/test_data.pkl'
with open(test_file, 'wb') as f:
    pickle.dump(testdata, f)

边的数量 466 520 0.8961538461538462
计算GRN的方法: cmi


In [ ]:
""" # test = False, 训练数据
# notebook 对进度条的显示支持不友好
# 处理一个文件需要50min, 若无必要不建议重新运行
batch, edge_percent = create_batch_dataset_from_cancer(filepath = filepath, 
                                test_pathway = test_pathway,
                                test = False) """

 Pathway: hsa00010, total contain 35 genes, and 35 links!:   0%|                        | 1/342 [00:02<16:19,  2.87s/it]

边的数量 35 35 1.0
计算GRN的方法: cmi


 Pathway: hsa00020, total contain 27 genes, and 43 links!:   1%|▏                       | 2/342 [00:04<12:54,  2.28s/it]

边的数量 44 43 1.0232558139534884
计算GRN的方法: cmi


 Pathway: hsa00030, total contain 19 genes, and 24 links!:   1%|▏                       | 3/342 [00:05<10:00,  1.77s/it]

...

边的数量 160 157 1.019108280254777
计算GRN的方法: cmi


 Pathway: hsa04933, total contain 47 genes, and 227 links!: 100%|████████████████████▉| 341/342 [50:00<00:06,  6.64s/it]

边的数量 204 227 0.8986784140969163
计算GRN的方法: cmi


 Pathway: hsa04934, total contain 56 genes, and 140 links!: 100%|█████████████████████| 342/342 [50:06<00:00,  8.79s/it]

边的数量 128 140 0.9142857142857143
计算GRN的方法: cmi


In [ ]:
""" # 处理后的batch保存到文件中, 无需运行
torch.save(batch,'Cancer_datasets/S33_Cancer_BRCA_processed_20250609.data') """

In [7]:
# 加载batch
batch = torch.load('Cancer_datasets/S33_Cancer_BRCA_processed_20250609.data')

In [ ]:
""" print(batch.x.shape,batch.num_nodes, batch.num_edges)
print(len(batch))
# 共285个图,对应KEGG中的285个key,其余的key与原exp的基因没有交集.
# 共14492个节点,29518条边
# print(edge_percent) 
# 边的比例为0.043041377101426546,稀疏图 """

torch.Size([14492, 100]) 14492 29518
285


In [ ]:
""" edge_percent = 0.043041377101426546
with open('reproducibility/train_batch.pkl','wb') as f:
    pickle.dump({'data': batch, 'edge_percent': edge_percent},f) """

### 3.2 Dignet.py
`load_data`
原文命名不规范, 其中调用create_batch_dataset_from_cancer返回的却不是Batch而是list, 本文已保存Batch数据,不用类型转换

In [42]:
train_file = 'reproducibility/train_batch.pkl'
test_file = 'reproducibility/test_data.pkl'

with open(test_file, 'rb') as f:
    d = pickle.load(f)
testdata = d # Data

with open(train_file, 'rb') as f:
    d = pickle.load(f)

data = d['data'] # Batch
edge_percent = d['edge_percent'] # float
if edge_percent < 0.1:
    edge_percent = 0.1

In [43]:
from torch_geometric.utils import to_dense_adj, to_dense_batch
import discrete.network_preprocess as network_preprocess
test_list = []
truelabel, node_mask, _ = network_preprocess.to_dense(testdata.x, testdata.edge_index,
                                                                    testdata.edge_attr,
                                                                    training=True, max_num_nodes=testdata.x.shape[0])
# PlaceHolder, Tensor
truelabel_discrete = truelabel.mask(node_mask, collapse=True)
truelabel_discrete = truelabel_discrete.E.squeeze(0) #(n,n,2)
test_list.append({'testdata': testdata, 'truelabel_discrete': truelabel_discrete}) #Data, Tensor

In [44]:
# 数据压缩, 从200多个细胞压缩为100个metacell,
# 此处又pca压缩到30个维度
use_PCA = 30
pca_file = 'reproducibility/pca_model_parameters.pkl'

pca = PCA(n_components=use_PCA)  # 设置维度为维度较小的数据集的维度
traindata_pca = pca.fit_transform(data.x.cpu().numpy())
with open(pca_file, 'wb') as file:
    pickle.dump(pca, file)
data.x = torch.tensor(traindata_pca, dtype=torch.float)
del traindata_pca
if test_list:
    for pcai in range(0, len(test_list)):
        testdata_pca = pca.transform(test_list[pcai]['testdata'].x.cpu().numpy())
        test_list[pcai]['testdata'].x = torch.tensor(testdata_pca, dtype=torch.float)
        del testdata_pca







训练见train.py

验证集见validation.py

注:Cancer_datasets/中的.data只含一种细胞的data, 只有一个dict,不符合输入格式

pathway/simulation/ 中的.data模拟数据, 是一个list,包含100个左右data(dict), 每个dict中有net,exp,符合输入格式

即使预训练模型和数据文件名相同, 效果也不好?

报错的函数路径

DigNet.train ->  

DigNet.load_data -> 

对.data文件,pathway.create_batch_dataset_simu

对.csv文件(选择这个),pathway.create_batch_dataset_from_cancer

->pathway.create_batch_dataset_from_cancer


-> cal_metacell

train_filename = 'Cancer_datasets\S33_Cancer_BRCA_output.csv'
(10015, 246)基因个数,细胞个数

return BRCA_exp_filter_saver:(基因个数,Cnum = 100)

<img src = 'validation.png'/>

## 4.其他文件
`pathway/PCA_CMI.py`
>conditional_mutual_info(X, Y, Z=np.array(1)):
计算条件互信息cmi,多元正态分布
如果X,Y是1维,修改形状为(-1,1)的2维数组.此处应该有误,因为cov是将每行当作特征,每列当作样本,(-1,1)的形状算不出协方差,(1,-1)的形状才能算出单个向量的方差.这里没出现报错,输入X,Y应该都是2维数组,列数相同(才能进行cov计算)
Z是常数或向量,用矩阵来说是0维或1维
分以下两种情况:
1.Z是0维,即Z不是一个条件,cmi退化成mi
d1,d2分别是X,Y的协方差的行列式
d3是X和Y拼接后的协方差的行列式
cmi = (1 / 2) * np.log((d1 * d2) / d3)
2.Z是1维,即Z是条件
d1是X,Z的协方差的行列式
d2是Y,Z的协方差的行列式
d3是Z的协方差的行列式
d4是X,Y,Z的协方差的行列式
cmi = (1 / 2) * np.log((d1 * d2) / (d3 * d4))
关于公式推导见**补充知识**
return cmi

>remove_edges(predicted_graph, data, L, theta):
先从predicted_graph中获得边的数量initial_num_edges和所有边edges
遍历每条边edge:
neighbors1 = edge[0]的子节点集合
neighbors2 = edge[1]的子节点集合
neighbors = 以上两个集合取交集,即两个点的公共子节点集合
nhbrs = neighbors的拷贝
T = 公共子节点个数
如果T < L(阈值)或者edge的两个点相同(即edge是自环边),不进行操作
否则:
在data中根据edge的两个端点找到x,y,他们的形状是(-1,1)
K = 在nhbrs中选择L个的组合
1.如果L == 0:
cmiVal = conditionconditional_mutual_info(x.T, y.T)
这个转置(即(1,-1)的输入)也跳过了上面conditionconditional_mutual_info中不合理的判别
如果cmiVal < theta则删除这条边
2.否则:遍历K中的zgroup
2.1 zgroup中没有数据,则跳过
2.2 data中通过zgroup查找z,即影响xy互信息的条件
cmiVal = conditional_mutual_info(x.T, y.T, z.T)
在对zgroup的遍历中获得最大的cmi,如果小于theta则删除这条边
在对每条边进行判断是否删除的操作后,计算图中边的数量,如果比输入时减少了,return predicted_graph, False
否则return predicted_graph, True

>pca_cmi(data, new_net_bit, theta, max_order, show=False):
创建有向图predicted_graph,节点为data的index
new_net_bit中的两列TF,Gene分别是每条边的头节点,尾节点
L = 0
nochange = False
data = data.T
循环,不断增加L,直到nochange == True
这个循环隐含的假设是,随着L增大,条件越多,cmi值会越小,导致不断有边被删除,直到所有边的端点的cmi大于theta,不能被删除.
但是查找资料:条件越多,条件互信息不一定越小或越大,其变化取决于条件变量与目标变量之间的因果或依赖结构.也就是说cmi不是一个单调的结构,这一段也需要考察.
调用时L = 1,即删除操作只进行一次,也不用考察L更大时的单调性质了.
show参数默认是False,如果为True有一段打印信息,调试时可以使用.
返回邻接矩阵和图结构
return predicted_adjMatrix, predicted_graph

## 5.补充知识
### 5.1 互信息(mi)和条件互信息(cmi)
熵,联合熵,条件熵的定义
$H(x) = -\sum_x p(x)log(p(x))\\
H(x,y) = -\sum_{x}\sum_{y} p(x,y)log(p(x,y))\\
H(x|y) = H(x,y) - H(y)$ 
首先,证明互信息mi的熵表达式和概率表达式等价
`熵表达式`
$mi(x,y) = H(x) + H(y) - H(x,y)\\
=H(x)-H(x|y)= H(y) - H(y|x)$
`概率表达式`
$mi(x,y) = \sum_{x}\sum_{y} p(x,y)log(\frac{p(x,y)}{p(x)p(y)})$
证明:
$$\begin{align*}
mi(x,y) &= H(x) + H(y) - H(x,y) \\
&= -\sum_x p(x)log(p(x))-\sum_y p(y)log(p(y))+\sum_{x}\sum_{y} p(x,y)log(p(x,y)) \\
&= -\sum_x\sum_{y} p(x,y)log(p(x))-\sum_{x}\sum_y p(x,y)log(p(y))+\sum_{x}\sum_{y} p(x,y)log(p(x,y)) \\
&=\sum_{x}\sum_{y} p(x,y)log(\frac{p(x,y)}{p(x)p(y)})
\end{align*}$$

条件互信息cmi
$cmi(x,y|z) =  H(x|z) + H(y|z) - H(x,y|z)\\ = \sum_zp(z)mi(x,y|z)$

关于本文中计算cmi的方法,网上的解释:
条件互信息（CMI）与行列式之间存在一定的联系，但直接使用行列式计算CMI并不常见，需要根据具体情况分析。下面从理论和实际应用两个角度进行说明：


#### **1. 理论上的联系**
对于**多元高斯分布**，CMI可以通过协方差矩阵的行列式计算。假设随机变量 \( X, Y, Z \) 服从联合高斯分布，则：
\[
\text{CMI}(X;Y|Z) = \frac{1}{2} \ln \left( \frac{\det(\Sigma_{XZ}) \cdot \det(\Sigma_{YZ})}{\det(\Sigma_{Z}) \cdot \det(\Sigma_{XYZ})} \right)
\]
其中：
- \( \Sigma_{XZ} \) 是 \( X \) 和 \( Z \) 的协方差矩阵
- \( \Sigma_{YZ} \) 是 \( Y \) 和 \( Z \) 的协方差矩阵
- \( \Sigma_{Z} \) 是 \( Z \) 的协方差矩阵
- \( \Sigma_{XYZ} \) 是 \( X, Y, Z \) 的联合协方差矩阵

**推导思路**：
高斯分布的微分熵为 \( H(X) = \frac{1}{2} \ln \left( (2\pi e)^n \det(\Sigma_X) \right) \)，代入CMI的熵表达式：
\[
\text{CMI}(X;Y|Z) = H(X|Z) + H(Y|Z) - H(X,Y|Z)
\]
通过行列式的比值化简可得上述公式。


#### **2. 离散数据的CMI与行列式**
对于**离散数据**，CMI的定义基于概率分布，而非协方差矩阵，因此**无法直接使用行列式计算**。例如：
\[
\text{CMI}(X;Y|Z) = \sum_{x,y,z} P(x,y,z) \log \frac{P(x,y|z)}{P(x|z) \cdot P(y|z)}
\]
此时需要通过统计频率估计概率分布，而非行列式。

#### 总结
本文假设变量都服从多元高斯分布

In [ ]:
#计算互信息的代码,可以看出用两种方法得到的mi差别很大
#注,mutual_info_score仅适用于离散数据,基于行列式的计算方法仅适用于多元高斯分布,
#所以两种方法可比性不高,但是本文中两种方式都出现了,需要考察
from sklearn.metrics import mutual_info_score
x = np.random.randn(20)
y = np.random.randn(20)
#离散化
bins = np.linspace(min(x),max(x),21)
bins1 = np.linspace(min(y),max(y),21)
labels = list(range(20))
x_discrete = pd.cut(x,bins = bins,labels = labels,include_lowest=True)
y_discrete = pd.cut(y,bins = bins1,labels = labels,include_lowest=True)
mi1 = mutual_info_score(x_discrete.to_list(),y_discrete.to_list())
d1 = np.cov(x_discrete)
d2 = np.cov(y_discrete)
d3 = np.linalg.det(np.cov(x_discrete,y_discrete))
mi2 = 1/2 * np.log((d1 * d2)/d3)
mi1, mi2

(1.7388948450374015, 0.04718490304900292)

### 5.2 初始化与激活函数
discrete/models/train_metrics.py中,使用了两种初始化方法,kaiming初始化和xavier初始化.
两种方法的比较如下
#### 1. **Xavier 初始化**（Glorot 初始化）&#xA;

*   **适用激活函数**：Sigmoid、Tanh、Softmax 等近似线性激活函数。

*   **数学原理**：权重 $W$ 的方差满足：

$\text{Var}(W) = \frac{2}{\text{fan\_in} + \text{fan\_out}}$

其中 $\text{fan\_in}$ 和 $\text{fan\_out}$ 分别为输入和输出神经元数量。

#### 2. **Kaiming 初始化**（He 初始化）&#xA;



*   **适用激活函数**：ReLU、LeakyReLU、PReLU 等 ReLU 类激活函数。


*   **数学原理**：考虑到 ReLU 对负值区域的抑制，权重 $W$ 的方差调整为：


$\text{Var}(W) = \frac{2}{\text{fan\_in}}$

其中 $\text{fan\_in}$ 为输入神经元数量。


### 5.3 FiLM（Feature-wise Linear Modulation）
FiLM（Feature-wise Linear Modulation）是一种强大的条件式特征调制技术，通过学习对特征图进行动态的线性变换，使模型能够适应不同的输入条件或任务需求。该技术最早由DeepMind在2017年提出，现已广泛应用于计算机视觉、自然语言处理和强化学习等领域。


### **核心原理**
FiLM的核心思想是通过外部条件（如文本描述、环境变量等）生成两组参数：**缩放因子**（γ）和**偏移量**（β），然后对输入特征图进行线性调制：
$$
\text{FiLM}(x, γ, β) = γ \otimes x + β
$$
其中：
- $x$ 是输入特征图（通常是CNN或Transformer的中间激活）。
- $γ$ 和 $β$ 是通过条件信息计算得到的参数，维度通常与 $x$ 的通道数匹配。
- $\otimes$ 表示逐通道的乘法操作。

### **FiLM的优势**
1. **灵活性**：通过条件信息动态调整特征，适应不同场景（如风格迁移、少样本学习）。
2. **轻量级**：仅需额外学习少量参数（γ和β），计算开销低。
3. **可解释性**：γ和β可视为对特征的“增益”和“偏置”，直观反映条件对特征的影响。
4. **兼容性**：可无缝集成到现有的CNN、Transformer等架构中。


### **应用场景**
#### **1. 视觉任务**
- **条件图像生成**：根据文本描述生成图像（如FiLM-GAN）。
- **少样本学习**：通过类别嵌入调整特征提取器。
- **域适应**：通过域标签动态调整模型参数。

#### **2. 自然语言处理**
- **多任务学习**：为不同任务生成特定的γ和β。
- **情感分析**：根据情感标签调制文本特征。

#### **3. 强化学习**
- **基于环境条件的策略调整**：通过FiLM动态改变策略网络。


### **FiLM与其他归一化技术的对比**
| **技术**       | **操作方式**                            | **核心思想**                          | **应用场景**                  |
|----------------|----------------------------------------|---------------------------------------|-----------------------------|
| **BatchNorm**  | 对一批样本的同一通道归一化               | 减少内部协变量偏移                     | 图像分类、CNN                |
| **LayerNorm**  | 对单个样本的所有通道归一化               | 稳定RNN/Transformer的训练              | 序列建模                    |
| **InstanceNorm**| 对单个样本的每个通道单独归一化            | 保留样本内的风格信息                   | 风格迁移                    |
| **FiLM**       | 通过外部条件动态缩放和偏移特征           | 条件式特征调制                         | 条件生成、少样本学习         |


### **FiLM的变体与扩展**
1. **Adaptive FiLM**：根据输入动态调整γ和β的生成方式。
2. **FiLM++**：引入注意力机制增强条件编码。
3. **Squeeze-and-Excitation (SE)**：可视为FiLM的简化版本，通过全局池化生成通道权重。


### **总结**
FiLM通过特征级的线性调制，为深度学习模型提供了强大的条件适应能力。它不仅能提升模型在多任务、少样本等场景下的性能，还能增强模型的可解释性。作为一种轻量级且灵活的技术，FiLM已成为现代深度学习架构中的重要组件。

### 5.4 pytorch张量计算相关知识

矩阵乘法:c = a @ b
a:(bs, q, k, v)
b:(bs,1, v, p)
c:(bs,q, k,p)
原理:最后两个维度是矩阵维度,其他维度是批量维度,批量维度需要保证相同或者为1(广播机制), 矩阵维度需要保持a的列 = b的行.

d = a[None,None,:]
d:(1,1,bs,q,k,v)
None切片的作用与unsqueeze相同,用于添加1维度

## 其他内容
.gitignore配置文件增加*.data,*.mat,*.csv,忽略解压出来的大文件
用gitdesktop推送克隆，比vscode流畅

暂时没看出vb下界和ce的区别